In [1]:
import pandas as pd
import numpy as np
import zipfile
from io import BytesIO
import requests
import geopandas as gpd
import shapefile #pip install pyshp
import plotly.express as px

# **Importation des données**

On veut d'abord récupérer les contours des circonscriptions de 2017 pour ensuite faire des cartes. Il faut d'abord décompresser le fichier de data.gouv.fr On obtient ensuite un shapefile qu'on va convertir en geopandas DataFrame

In [2]:
url = 'https://static.data.gouv.fr/resources/carte-des-circonscriptions-legislatives-2012-et-2017/20170721-135755/france-circonscriptions-legislatives-2012.zip'
print("debut téléchargement")
req = requests.get(url)
zf= zipfile.ZipFile(BytesIO(req.content))
print("fin téléchargement")
name='france-circonscriptions-legislatives-2012.shp' #On veut récupérer ce fichier
print("début extraction")
zf.extractall()
tab=gpd.read_file(name)
print("fin extraction")
tab

debut téléchargement
fin téléchargement
début extraction
fin extraction


,ID,code_dpt,code_reg,nom_dpt,nom_reg,num_circ,geometry
0,33004,33,75,GIRONDE,AQUITAINE-LIMOUSIN-POITOU-CHARENTES,4,"POLYGON ((-0.45495 44.95342, -0.40932 44.94761..."
1,38001,38,84,ISERE,AUVERGNE-RHONE-ALPES,1,"POLYGON ((5.80529 45.20620, 5.75468 45.19679, ..."
2,59010,59,32,NORD,NORD-PAS-DE-CALAIS-PICARDIE,10,"POLYGON ((3.05875 50.78071, 3.08067 50.77286, ..."
3,33007,33,75,GIRONDE,AQUITAINE-LIMOUSIN-POITOU-CHARENTES,7,"POLYGON ((-0.61065 44.82247, -0.60305 44.81848..."
4,ZA001,ZA,01,GUADELOUPE,GUADELOUPE,1,"MULTIPOLYGON (((-61.49348 16.35364, -61.45097 ..."
...,...,...,...,...,...,...,...
561,ZX001,ZX,-,SAINT-MARTIN/SAINT-BARTHELEMY,-,1,"MULTIPOLYGON (((-63.09622 18.06261, -63.08109 ..."
562,ZW001,ZW,-,WALLIS-ET-FUTUNA,-,1,"MULTIPOLYGON (((-176.25085 -13.29059, -176.224..."
563,ZP001,ZP,-,POLYNESIE-FRANCAISE,-,1,"MULTIPOLYGON (((-150.65198 -17.66457, -150.628..."
564,ZP002,ZP,-,POLYNESIE-FRANCAISE,-,2,"MULTIPOLYGON (((-153.03842 -22.64486, -153.033..."


**Importation des données dans Jupypter Notebook**
*Il y a un bug avec geopandas dans le notebook donc je vais plutôt importer manuellement les données dans Collab et puis les lire directement avec pandas. Néanmoins, le passage suivant doit fonctionner dans un notebook*


In [3]:
caract_circo=pd.read_csv("C:/Users/Paul/Documents/GitHub/projetpython/1. Rassemblement des données/basefinale.csv",";")
caract_deputes=pd.read_csv("C:/Users/Paul/Documents/GitHub/projetpython/1. Rassemblement des données/caract_deputes.csv",";")
participation_deputes=pd.read_csv("C:/Users/Paul/Documents/GitHub/projetpython/1. Rassemblement des données/participation.csv",";")
twitter_deputes=pd.read_csv("C:/Users/Paul/Documents/GitHub/projetpython/1. Rassemblement des données/stats_twitter.csv",";")


FileNotFoundError: ignored

**Dans Collab** *Il faut au préalable importer manuellement les csv (ce que je fais)*

In [4]:
caract_circo=pd.read_csv("basefinale.csv",";")
caract_deputes=pd.read_csv("caract_deputes.csv",";")
participation_deputes=pd.read_csv("participation.csv",";")
twitter_deputes=pd.read_csv("stats_twitter.csv",";")

**Ce qui suit est valable quel que soit le mode d'importation choisi**

In [5]:
tab2=caract_deputes
tab["code_dpt"]=tab["code_dpt"].astype(str)
tab["num_circ"]=tab["num_circ"].astype(str)
tab2["num_deptmt"]=tab2["num_deptmt"].astype(str)
tab2["num_circo"]=tab2["num_circo"].astype(str)
tab_final=tab.merge(tab2, how='inner',left_on=['code_dpt','num_circ'], right_on = ['num_deptmt','num_circo'])
tab3=twitter_deputes
tab_final=tab3.merge(tab_final,on='nom')
tab_final2 = gpd.GeoDataFrame(tab_final)
tab_final2

,Unnamed: 0,Tweets,Followers,Following,Followers ratio,Listed,Replies/100derniers,Tweets with @mentions/100derniers,Tweets with #hashtags/100derniers,Retweets/100,Tweets with links/100derniers,Tweets with media/100derniers,Mot prefere,twitter_x,date_scrap,nom,DEPSIR_x,ID,code_dpt,code_reg,nom_dpt,nom_reg,num_circ,geometry,nom_de_famille,prenom,sexe,date_naissance,num_deptmt,nom_circo,num_circo,mandat_debut,mandat_fin,ancien_depute,groupe_sigle,parti_ratt_financier,autres_mandats,anciens_autres_mandats,nb_mandats,twitter_y,ville_naissance,dep_naissance,profession,cat_soc_pro,patrimoine,ancrage_local,DEPSIR_y
0,0,3681.0,5662.0,1023.0,5.53,131.0,1.0,55.0,49.0,8.0,92.0,0.0,modle,CedricRoussel06,01/01/2022,Cédric Roussel,0603,06003,06,93,ALPES-MARITIMES,PROVENCE-ALPES-COTE D'AZUR,3,"POLYGON ((7.29624 43.75871, 7.36923 43.73972, ...",Roussel,Cédric,1,1972-10-10,06,Alpes-Maritimes,3,2017-06-21,NaN,0,LREM,La République en Marche,NaN,NaN,1,CedricRoussel06,Brest,Finistère,Conseiller en gestion de patrimoine indépendant,Professions libérales et assimilés,https://www.hatvp.fr/pages_nominatives/roussel...,0,0603
1,1,4956.0,4653.0,1215.0,3.83,82.0,5.0,60.0,13.0,15.0,82.0,1.0,cest,f_personne,01/01/2022,Pascale Fontenel-Personne,7203,72003,72,52,SARTHE,PAYS DE LA LOIRE,3,"POLYGON ((0.61443 47.69421, 0.59334 47.67184, ...",Fontenel-Personne,Pascale,0,1962-05-26,72,Sarthe,3,2017-06-21,NaN,0,MODEM,La République en Marche,NaN,NaN,1,f_personne,Le Mans,Sarthe,Industriel-Chef d'entreprise,Commerçants et assimilés,https://www.hatvp.fr/pages_nominatives/fontene...,1,7203
2,2,6304.0,10539.0,1264.0,8.34,178.0,1.0,95.0,21.0,92.0,8.0,14.0,cest,guillaumekasba,01/01/2022,Guillaume Kasbarian,2801,28001,28,24,EURE-ET-LOIR,CENTRE-VAL DE LOIRE,1,"POLYGON ((1.58293 48.76756, 1.62496 48.74903, ...",Kasbarian,Guillaume,1,1987-02-28,28,Eure-et-Loir,1,2017-06-21,NaN,0,LREM,La République en Marche,NaN,NaN,1,guillaumekasba,Marseille,Bouches-du-Rhône,Cadre supérieur (secteur privé),Cadres d'entreprise,https://www.hatvp.fr/pages_nominatives/kasbari...,0,2801
3,3,6427.0,2594.0,1287.0,2.02,113.0,7.0,73.0,15.0,45.0,44.0,15.0,sant,Cisaacsibille,01/01/2022,Cyrille Isaac-Sibille,6912,69012,69,84,RHONE,AUVERGNE-RHONE-ALPES,12,"POLYGON ((4.76121 45.77069, 4.79246 45.77329, ...",Isaac-Sibille,Cyrille,1,1958-04-30,69,Rhône,12,2017-06-21,NaN,0,MODEM,Mouvement Démocrate,Sainte-Foy-lès-Lyon / Conseil municipal / membre,NaN,1,Cisaacsibille,Lyon 6,Rhône,Médecin,Professions libérales et assimilés,https://www.hatvp.fr/pages_nominatives/isaac-s...,1,6912
4,4,3065.0,3773.0,812.0,4.65,130.0,6.0,77.0,51.0,46.0,53.0,3.0,cest,g_vuilletet,01/01/2022,Guillaume Vuilletet,9502,95002,95,11,VAL-D'OISE,ILE-DE-FRANCE,2,"POLYGON ((2.35930 49.14735, 2.37316 49.15933, ...",Vuilletet,Guillaume,1,1967-06-20,95,Val-d'Oise,2,2017-06-21,NaN,0,LREM,La République en Marche,NaN,NaN,1,g_vuilletet,Beauvais,Oise,Cadre supérieur (secteur privé),Cadres d'entreprise,https://www.hatvp.fr/pages_nominatives/vuillet...,0,9502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,557,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NON,0,Luc Lamirault,2803,28003,28,24,EURE-ET-LOIR,CENTRE-VAL DE LOIRE,3,"POLYGON ((1.02960 48.13284, 0.88890 48.10227, ...",Lamirault,Luc,1,1962-05-12,28,Eure-et-Loir,3,2021-01-22,NaN,0,AE,Mouvement Démocrate,"/ Conseil municipal / adjoint au maire , eure...",NaN,2,NaN,nc,NaN,chef d'entreprise,Chefs d'entreprise de 10 salariés ou plus,https://www.hatvp.fr/pages_nominatives/lamirau...,0,2803
522,558,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NON,0,Christelle Petex-Levet,7403,74003,74,84,HAUTE-SAVOIE,AUVERGNE-RHONE-ALPES,3,"POLYGON ((6.35472 46.22969, 6.38989 46.24966, ...",Petex-Levet,Christelle,0,1980-09-02,74,Haute-Savoie,3,2021-08-01,NaN,0,LR,Les Républicains,NaN,NaN,1,NaN,Annecy,Haute-Savoie,NaN,NaN,https://www.hatvp.fr/pages_nominatives/petex-l...,1

*Nous réalisons maintenant des cartes à partir de ce tableau. Il manque quelques lignes dans le tableau donc tout l'hexagone ne sera pas coloré. On se restreint à la France Métropolitaine par souci de lisibilité.*

# **Cartes sur les caractéristiques des députés**

Le sexe de nos députés

In [6]:
fig = px.choropleth_mapbox(tab_final2,mapbox_style="carto-positron",
                   geojson=tab_final2.geometry,color='sexe',locations=tab_final2.index,center={"lat": 46.45200710842239, "lon": 2.997362843386706},zoom=4,hover_name='nom',title="La sexe de nos députés")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

L'ancrage local de nos députés. Ceux qui sont dans nés dans le département de leur circonscription. *Il semble y avoir un meilleur ancrage local dans les départements ruraux. Il faudra regarder les conséquences de cet ancrage sur l'action de nos députés*

In [7]:
fig = px.choropleth_mapbox(tab_final2,mapbox_style="carto-positron",
                   geojson=tab_final2.geometry,color='ancrage_local',locations=tab_final2.index,center={"lat": 46.45200710842239, "lon": 2.997362843386706},zoom=4,hover_name='nom',title="L'ancrage local de nos députés")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# **Cartes sur Twitter**

La France du nombre de tweets selon nos députés

In [8]:
fig = px.choropleth_mapbox(tab_final2,mapbox_style="carto-positron",
                   geojson=tab_final2.geometry,color='Tweets',locations=tab_final2.index,center={"lat": 46.45200710842239, "lon": 2.997362843386706},zoom=4,hover_name='nom',title="La France du nombre de tweets selon nos députés")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

La France du nombre de Retweets sur les 100 derniers tweets de nos députés *Les députés qui retweetent le plus sont ceux qui crééent le moins de contenu original. Pour le coup, on observe une grande disparité sur le territoire. Il sera intéressant d'essayer de prédire cette variable afin de voir ce qui la détermine*

In [9]:
fig = px.choropleth_mapbox(tab_final2,mapbox_style="carto-positron",
                   geojson=tab_final2.geometry,color='Retweets/100',locations=tab_final2.index,center={"lat": 46.45200710842239, "lon": 2.997362843386706},zoom=4,hover_name='nom',title="La France du nombre de Retweets sur les 100 derniers tweets de nos députés")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

La France du nombre de Followers de nos *Il y a une assez grande homogénéité sur le territoire sauf quelques "célébrités locales qui sont en fait des célébrités nationales: Marine Le Pen, Jean-Luc Mélenchon, Christophe Castaner, François Ruffin, ...*

In [10]:
fig = px.choropleth_mapbox(tab_final2,mapbox_style="carto-positron",
                   geojson=tab_final2.geometry,color='Followers',locations=tab_final2.index,center={"lat": 46.45200710842239, "lon": 2.997362843386706},zoom=4,hover_name='nom',title="La France du nombre de Followers de nos députés")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()